In [1]:
from load_data import get_sample
from PIL import Image
from stablediffusion import StableDiffusion
import torch
from vision_encoder import vision_encoder
from llm import ClipCaptionModel, generate_text_with_gumbel_softmax
import os
from transformers import GPT2Tokenizer, AdamW, CLIPProcessor, CLIPModel
import numpy as np
from fashion_clip.fashion_clip import FashionCLIP
from mapping import get_gpt2_logits, map_prompt_to_clip, recover_text_from_one_hot
from custom_clip import CustomCLIPTextModel 
from torchvision import transforms
import random
import gc

def set_seed(seed):
    """Set all seeds to make computations deterministic where possible."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Seed all GPUs for multi-GPU setups
    os.environ['PYTHONHASHSEED'] = str(seed)

    # Enable deterministic behavior in PyTorch (may impact performance)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

our_clip = CustomCLIPTextModel.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="text_encoder")
sd = StableDiffusion()
encoder = vision_encoder()
fclip = FashionCLIP('fashion-clip')
CPU = torch.device('cpu')
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

clip_vit = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_vit.to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

for param in clip_vit.parameters():
    param.requires_grad = False

for param in sd.parameters():
    param.requires_grad = False

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
prefix_length = 10
current_directory = os.getcwd()
save_path = os.path.join(current_directory, "saved_models")
os.makedirs(save_path, exist_ok=True)

model = ClipCaptionModel(prefix_length)
model_path = os.path.join(save_path, 'fashion.pt')
model.load_state_dict(torch.load(model_path, map_location=CPU)) 
model = model.eval() 
model = model.to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [3]:
def compute_clip_loss(pil_image, output_tensor):
    def prepare_image(image, device, is_tensor=False):
        if is_tensor:
            # Apply normalization directly to tensor
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            image = transform(image)  # Assumes image is already a tensor
        else:
            # Convert PIL Image to tensor and apply transformations
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            image = transform(image).to(device)
        return image.unsqueeze(0)

    # Process the PIL image and the output tensor
    pil_tensor = prepare_image(pil_image, device)
    output_tensor = output_tensor.permute(0, 1, 2)  # Convert to [C, H, W] if not already
    output_tensor = prepare_image(output_tensor, device, is_tensor=True)

    # Get CLIP features
    features_pil = clip_vit.get_image_features(pixel_values=pil_tensor)
    features_output = clip_vit.get_image_features(pixel_values=output_tensor)

    # Calculate MSE Loss
    mse_loss = torch.nn.MSELoss()
    loss = mse_loss(features_pil, features_output)

    return loss

In [4]:
for name, parameter in model.named_parameters():
    param.requires_grad = True  # Ensure all parameters are trainable

In [ ]:
# Set up the optimizer with only those parameters that need to be updated
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

# Example loop for processing and training
num_training_steps = 100
for step in range(num_training_steps):
    pil_image, row = get_sample(0)

    # Encode images and prepare embeddings
    image_embeddings = fclip.encode_images([pil_image], batch_size=1)
    image_embeddings /= np.linalg.norm(image_embeddings, ord=2, axis=-1, keepdims=True)
    image_embeddings = torch.tensor(image_embeddings, device=device, dtype=torch.float32)

    # Ensure gradients are computed for embeddings part
    image_embeddings.requires_grad_(True)

    prefix_embed = model.clip_project(image_embeddings).reshape(1, prefix_length, -1)
    set_seed(42)
    out, soft_tokens_list = generate_text_with_gumbel_softmax(model, tokenizer, embed=prefix_embed, temperature=0.1)
    
    print(f"Prompt: {out}")

    combined_softmax_outputs = torch.cat(soft_tokens_list, dim=0)
    tokens, attention_mask, input_ids, out_prompt = map_prompt_to_clip(combined_softmax_outputs)
    token_embeddings = our_clip(inputs_embeds=tokens.unsqueeze(0), attention_mask=attention_mask, input_ids = input_ids.unsqueeze(0))[0]
    sd.requires_grad_(False)
    output = sd([out_prompt], tokens=token_embeddings.to(device), num_inference_steps=1)
    loss = compute_clip_loss(pil_image, output)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Step {step} Loss: {loss.item()}")
    
    del pil_image, row, image_embeddings, prefix_embed, out, combined_softmax_outputs, tokens, attention_mask, input_ids, token_embeddings, output, loss
    torch.cuda.empty_cache()  # Clear CUDA cache
    gc.collect()
    
    print(f"Step {step}: Gradients for LLM (model)")


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  1.78it/s]


Prompt: Body in crushed velvet with thin shoulder straps and a narrow crossover fastening at the back. Narrow, adjustable shoulder straps and spaghetti straps at the back. Unlined. The polyester content of the bra is partly recycled. For the matching sweatshirt, search on 02224.2.Fully lined. For other matching sweats


/home/ec2-user/SageMaker/llm_diffusion/mapping.py:59: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /opt/conda/conda-bld/pytorch_1696595231861/work/torch/csrc/utils/tensor_new.cpp:605.)
  transformation_matrix = torch.sparse.FloatTensor(indices, values, torch.Size([gpt2_vocab_size, clip_vocab_size]))


  0%|          | 0/1 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


...

100%|██████████| 1/1 [00:00<00:00, 15.38it/s]


Prompt:  Short shorts in spaghetti belt belt, spaghetti belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt belt


  0%|          | 0/1 [00:00<?, ?it/s]

Step 49 Loss: 0.22835630178451538
Step 49: Gradients for LLM (model)


100%|██████████| 1/1 [00:00<00:00, 15.38it/s]


In [ ]:
def show_image(output):
    output_image = (output.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
    output_image = Image.fromarray(output_image)
    output_image.show()

show_image(output)

In [ ]:
import torch

# Ensure CUDA is available
if torch.cuda.is_available():
    # Select your CUDA device, generally 0 by default for single-GPU setups
    device = torch.device('cuda:0')
    
    # Get GPU details
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)
    max_allocated_memory = torch.cuda.max_memory_allocated(device)
    max_cached_memory = torch.cuda.max_memory_reserved(device)

    # Convert bytes to gigabytes for easier interpretation
    gb_divisor = 1024**3

    print(f"Total Memory: {total_memory / gb_divisor:.2f} GB")
    print(f"Allocated Memory: {allocated_memory / gb_divisor:.2f} GB")
    print(f"Cached Memory: {cached_memory / gb_divisor:.2f} GB")
    print(f"Max Allocated Memory: {max_allocated_memory / gb_divisor:.2f} GB")
    print(f"Max Cached Memory: {max_cached_memory / gb_divisor:.2f} GB")
else:
    print("CUDA is not available. Check your installation or GPU availability.")
